# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [1]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests


# Import API key
from api_keys import geoapify_key

In [2]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("output_data/cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,ishigaki,24.3448,124.1572,303.22,79,75,3.09,JP,1721064488
1,1,puerto natales,-51.7236,-72.4875,279.40,81,75,2.06,CL,1721064490
2,2,edinburgh of the seven seas,-37.0676,-12.3116,286.69,82,61,13.46,SH,1721064314
3,3,ewo,-0.8725,14.8206,303.73,39,85,1.50,CG,1721064492
4,4,adamstown,-25.0660,-130.1015,292.20,67,85,2.78,PN,1721064493


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [3]:
%%capture --no-display

# Configure the map plot
map1 = city_data_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    tiles = "OSM",
    frame_width = 900,
    frame_height = 700,
    size = "Humidity",
    scale = 0.5,
    color = "City",
    fill_alpha = 0.5,
    title = "Cities in City_data_df",
    xlabel = "Latitude",
    ylabel = "Longitude")

# Display the map
map1


:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (City,Humidity)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [4]:
# Narrow down cities that fit criteria and drop any results with null values

# Filter the city_data_df DataFrame for my family's ideal weather for vacationing
ideal_weather_df = city_data_df.loc[
    (city_data_df["Humidity"] <= 70) &
    (city_data_df["Max Temp"] >= 273) &
    (city_data_df["Max Temp"] <= 297) &
    (city_data_df["Wind Speed"] <= 10) &
    (city_data_df["Cloudiness"] <= 50) 
    ]

# Drop any rows with null values
ideal_weather_df.dropna().reset_index(drop=True)

# Display sample data
ideal_weather_df

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
9,9,iqaluit,63.7506,-68.5145,287.00,54,40,1.54,CA,1721064499
12,12,solwezi,-12.1688,26.3894,291.52,32,6,1.54,ZM,1721064502
23,23,port alfred,-33.5906,26.8910,287.47,65,1,7.05,ZA,1721064515
43,43,vizinga,61.0749,50.1031,292.74,56,1,0.69,RU,1721064539
47,47,upington,-28.4478,21.2561,287.36,30,0,1.03,ZA,1721064543
...,...,...,...,...,...,...,...,...,...,...
576,576,north bend,43.4065,-124.2243,292.68,64,43,5.58,US,1721065170
581,581,menongue,-14.6585,17.6910,290.58,11,6,1.99,AO,1721065176
584,584,berezovo,63.9381,65.0419,289.41,70,0,1.91,RU,1721065180
588,588,tsiroanomandidy,-18.7667,46.0333,289.22,42,2,0.83,MG,1721065185


### Step 3: Create a new DataFrame called `hotel_df`.

In [19]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df = ideal_weather_df[["City", "Country", "Lat", "Lng", "Humidity"]].copy()

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df["Hotel Name"] = ""

# Display sample data
hotel_df

,City,Country,Lat,Lng,Humidity,Hotel Name
9,iqaluit,CA,63.7506,-68.5145,54,
12,solwezi,ZM,-12.1688,26.3894,32,
23,port alfred,ZA,-33.5906,26.8910,65,
43,vizinga,RU,61.0749,50.1031,56,
47,upington,ZA,-28.4478,21.2561,30,
...,...,...,...,...,...,...
576,north bend,US,43.4065,-124.2243,64,
581,menongue,AO,-14.6585,17.6910,11,
584,berezovo,RU,63.9381,65.0419,70,
588,tsiroanomandidy,MG,-18.7667,46.0333,42,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [ ]:
# Set parameters to search for a hotel
radius = 10000
categories = "accommodation.hotel"
params = {
    "categories":categories,
    "apiKey":geoapify_key 
}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # Get latitude, longitude from the DataFrame
    latitude = hotel_df.loc[index, "Lat"]
    longitude = hotel_df.loc[index, "Lng"]

    # Add the current city's latitude and longitude to the params dictionary
    params["filter"] = f"circle:{longitude},{latitude},{radius}"
    params["bias"] = f"proximity:{longitude},{latitude}"

    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"

    # Make and API request using the params dictionary
    name_address = requests.get(base_url, params=params)

    # Convert the API response to JSON format
    name_address_json = name_address.json()

    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address_json["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"

    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df

Starting hotel search
port-aux-francais - nearest hotel: Keravel
ilulissat - nearest hotel: Best Western Plus Hotel Ilulissat
somerset east - nearest hotel: No hotel found
waitangi - nearest hotel: Hotel Chathams
blackmans bay - nearest hotel: Villa Howden
egilsstadir - nearest hotel: Valaskjálf
nova granada - nearest hotel: Hotel Granada
praia da vitoria - nearest hotel: Salles
yellowknife - nearest hotel: No hotel found
polyarnyy - nearest hotel: No hotel found
ushuaia - nearest hotel: Apart Hotel Aires del Beagle
aasiaat - nearest hotel: SØMA
ambalangoda - nearest hotel: Hotel Italia
betio village - nearest hotel: Seaman Hostel
carnarvon - nearest hotel: No hotel found
acapulco de juarez - nearest hotel: Hotel del Valle
tiksi - nearest hotel: Арктика
cole harbour - nearest hotel: Hearthstone Inn Boutique Hotel Dartmouth/Halifax
kapuskasing - nearest hotel: Travelodge
diffa - nearest hotel: UNHCR guest house
constantia - nearest hotel: Hotel Dali
calvinia - nearest hotel: No hotel fo

KeyboardInterrupt: 

### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [ ]:
%%capture --no-display

# Configure the map plot
map2 = hotel_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    tiles = "OSM",
    frame_width = 900,
    frame_height = 700,
    size = "Humidity",
    scale = 0.5,
    color = "City",
    fill_alpha = 0.5,
    title = "My Family's Ideal Cities to Vacation",
    xlabel = "Latitude",
    ylabel = "Longitude",
    hover_cols = ["Hotel Name", "Country"]
)

# Display the map
map2

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (City,Humidity,Hotel Name,Country)